## **2.1 Carga datos**

In [1]:
import pandas as pd

df_balanced_sent = pd.read_csv('/content/drive/MyDrive/NLP/df_balanced_sent.csv')

In [55]:
print(df_balanced_sent.head())

                                          reviewText  sentiment
0  It's got a lot more kick than what you'll find...          1
1  Cookies seemed a little old, did not come with...          0
2  These caramels are grainy and tasteless. They ...          0
3                                 very good , strong          1
4  Not as good as home made from scratch, but not...          1


## **2.2 Función de preprocesado completa**

In [45]:
! pip install nltk

In [46]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
!pip install num2words

In [70]:
import unicodedata
from num2words import num2words
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist

def preprocess(text):

  # Eliminamos acentos y signos de puntuación
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

  # Tokenizamos y convertimos a minúsculas
  #tokenizer = RegexpTokenizer('r\w+')
  tokenized_text = word_tokenize(text.lower())

  # Eliminamos signos de puntuación y caracteres especiales
  tokenized_text = [word for word in tokenized_text if word.isalnum()]

  # Eliminamos stop words
  stop_words = set(stopwords.words('english'))
  tokenized_text = [word for word in tokenized_text if word not in stop_words]

  # Lemmatización
  lemmatizer = WordNetLemmatizer()
  tokenized_text = [lemmatizer.lemmatize(word).strip() for word in tokenized_text]

  # Convertimos números a palabras
  for i, word in enumerate(tokenized_text):
    if word.isdigit():
      tokenized_text[i] = num2words(int(word), lang='en')

  # Vamos a eliminar también las palabras más frecuentes, ya que no nos aportarán valor. Son palabras como vimos en el EDA como 'product', 'taste, 'flavour', que se usan en contexto
  # tanto positivo como negativo
  word_freq = FreqDist(tokenized_text)

  most_common_words = set(word for word, freq in word_freq.most_common(5)) # Tras hacer pruebas, parece que eliminar las 5 más usadas da resultados razonables
  #tokenized_text = [word for word in tokenized_text if word not in most_common_words]

  # Unimos el resultado en un texto limpio
  clean_text = ' '.join(tokenized_text)

  return clean_text


## **2.3 Aplicación del preprocesado al corpus**

In [71]:
df_balanced_sent['processed_reviews'] = df_balanced_sent['reviewText'].apply(preprocess)

In [72]:
print(f"Review original: {df_balanced_sent['reviewText'].values[0]}")
print(f"Review procesada: {df_balanced_sent['processed_reviews'].values[0]}")

Review original: It's got a lot more kick than what you'll find at your local grocery store.
I'd give it a solid 5 stars if it had a bit less sugar (and fewer calories).
I've yet to find a ginger beer as strong as the stuff I had as a kid, but this is the best I've found so far.
It's nothing like ginger ale, I don't know what the other guy was smoking.
Review procesada: got lot kick find local grocery store give solid five star bit le sugar fewer calorie yet find ginger beer strong stuff kid best found far nothing like ginger ale know guy smoking


In [74]:
# Guardamos el dataset con el nuevo estado para pasar al modelado.

from google.colab import drive

drive.mount('/content/drive')

df_balanced_sent.to_csv('/content/drive/MyDrive/NLP/df_processed.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **2.4 Conclusiones**

Hemos ido construyendo la **función de preprocesad**o, haciendo diferentes pruebas.

El **proceso** a seguir ha sido:
  - **Eliminar tildes y signos de puntuación**.
  - **Tokenizar y dejar en minúscula**s (ya dejamos en minúsculas anteriormente para ver las nubes de palabras y explorar el texto pero por si acaso aplicamos nuevamente).
  - **Eliminamos las stop words** según el diccionario de stopwords en **inglés** de nltk.
  - Realizamos la **lemmatización** y **eliminamos espacios extra**.
  - Convertimos los posibles **números a palabras**.
  - Eliminar las palabras más frecuentes. Agregamos esto en una nueva iteración, ya que vemos que las reviews resultantes del procesado incluyen demasiadas palabras que aparentemente no aportan valor, como por ejemplo "Product". Decidimos eliminar las 5 más comunes (ver nubes de palabras y otras representaciones en el apartado de EDA para obtener más información al respecto)
  